In [8]:
import pandas as pd
from sodapy import Socrata
from uszipcode import SearchEngine
import pickle

In [9]:
pd.options.display.max_columns=100

In [10]:
client = Socrata("data.lacity.org", None)

results = client.get("7fvc-faax", limit=2000000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [11]:
results_df.head()

,:@computed_region_2dna_qi2s,:@computed_region_k96s_3jcv,:@computed_region_kqwf_mjcx,:@computed_region_qz3q_ghft,:@computed_region_tatf_ua23,:@computed_region_ur2y_g4cx,area_id,area_name,crm_cd,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,crm_cd_desc,cross_street,date_occ,date_rptd,dr_no,location,location_1,mocodes,premis_cd,premis_desc,rpt_dist_no,status,status_desc,time_occ,vict_age,vict_descent,vict_sex,weapon_desc,weapon_used_cd
0,26,626,7,23667,1086,NaN,07,Wilshire,930,930,NaN,NaN,NaN,CRIMINAL THREATS - NO WEAPON DISPLAYED,NaN,2010-03-12T00:00:00.000,2010-03-12T00:00:00.000,100700660,7600 BEVERLY BL,"{'type': 'Point', 'coordinates': [-118.3555, 3...",0421,109,PARK/PLAYGROUND,0724,AA,Adult Arrest,1050,41,B,M,VERBAL THREAT,511
1,26,635,7,23667,332,NaN,07,Wilshire,626,626,NaN,NaN,NaN,INTIMATE PARTNER - SIMPLE ASSAULT,NaN,2010-03-13T00:00:00.000,2010-03-13T00:00:00.000,100700661,600 S DETROIT ST,"{'type': 'Point', 'coordinates': [-118.3451, 3...",0400 2000,502,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",0735,AA,Adult Arrest,0600,27,O,F,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",400
2,26,633,6,23679,809,NaN,07,Wilshire,930,930,998,NaN,NaN,CRIMINAL THREATS - NO WEAPON DISPLAYED,3RD ST,2010-03-23T00:00:00.000,2010-03-24T00:00:00.000,100700688,LA CIENEGA BL,"{'type': 'Point', 'coordinates': [-118.3765, 3...",0400 0416 0421 1402,108,PARKING LOT,0721,AA,Adult Arrest,2350,20,H,F,VERBAL THREAT,511
3,52,612,12,23080,1229,NaN,07,Wilshire,626,626,NaN,NaN,NaN,INTIMATE PARTNER - SIMPLE ASSAULT,NaN,2010-03-23T00:00:00.000,2010-03-24T00:00:00.000,100700689,1100 NORTON AV,"{'type': 'Point', 'coordinates': [-118.3219, 3...",0400 0416 2000,502,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",0759,AA,Adult Arrest,2330,35,O,M,UNKNOWN WEAPON/OTHER WEAPON,500
4,69,651,12,23080,1105,NaN,07,Wilshire,236,236,NaN,NaN,NaN,INTIMATE PARTNER - AGGRAVATED ASSAULT,NaN,2010-04-05T00:00:00.000,2010-04-05T00:00:00.000,100700719,4400 W WASHINGTON BL,"{'type': 'Point', 'coordinates': [-118.3353, 3...",0400 0416 2000,502,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",0787,AO,Adult Other,2000,43,K,F,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",400


In [ ]:
# with open('results_df.pickle', 'wb') as f:
#     # Pickle the 'data' dictionary using the highest protocol available.
#     pickle.dump(results_df, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open('results_df.pickle', 'rb') as f:
#     # The protocol version used is detected automatically, so we do not
#     # have to specify it.
#     results_df = pickle.load(f)

In [ ]:
list(results_df)

In [11]:
location = results_df['location_1'].head()

In [17]:
location.head()

0    {'type': 'Point', 'coordinates': [-118.3555, 3...
1    {'type': 'Point', 'coordinates': [-118.3451, 3...
2    {'type': 'Point', 'coordinates': [-118.3765, 3...
3    {'type': 'Point', 'coordinates': [-118.3219, 3...
4    {'type': 'Point', 'coordinates': [-118.3353, 3...
Name: location_1, dtype: object

In [21]:
location[0]

{'type': 'Point', 'coordinates': [-118.3555, 34.0761]}

In [22]:
type(location[0])

dict

In [20]:
coordinates = location[0]['coordinates']
type(coordinates)

list

In [23]:
coordinates

[-118.3555, 34.0761]

In [24]:
coordinates[0]

-118.3555

In [25]:
coordinates[1]

34.0761

In [59]:
rows=len(results_df)
for x in range(rows):
    results_df['lat'] = results_df['location_1'][x]['coordinates'][0]
    results_df['lon'] = results_df['location_1'][x]['coordinates'][1]
results_df.head()

KeyboardInterrupt: 